# Reorganizing the Data

In [1]:
import pandas as pd
import os

servers = ["Server_1", "Server_2", "Server_3", "Server_4", "Server_5", "Server_6"]

days = ["Day_1", "Day_2", "Day_3", "Day_4", "Day_5", "Day_6", "Day_7", "Day_8", "Day_9", "Day_10", "Day_11", "Day_12", "Day_13", "Day_14"]

columns = ["node_load1",
           "node_memory_MemAvailable_bytes",
           "node_network_transmit_bytes_total{device=\"eno1\"}"]

for server in servers:
    for day in days:
        if not os.path.exists(f"{server}_Training_Sets/{day}.xlsx"):
            server_metrics_df = pd.read_csv(f"Training_Sets/Training_Set_{day}/physical_level/{server.lower()}.csv", usecols=columns, sep=";")

            for column in columns:
                server_metrics_df[column] = pd.to_numeric(server_metrics_df[column], errors='coerce')

            server_metrics_df.to_excel(f"{server}_Training_Sets/{day}.xlsx", index=False)

/tmp/ipykernel_170681/2019955214.py:15: DtypeWarning: Columns (852,882,1413) have mixed types. Specify dtype option on import or set low_memory=False.
  server_metrics_df = pd.read_csv(f"Training_Sets/Training_Set_{day}/physical_level/{server.lower()}.csv", usecols=columns, sep=";")
/tmp/ipykernel_170681/2019955214.py:15: DtypeWarning: Columns (852,882,1413) have mixed types. Specify dtype option on import or set low_memory=False.
  server_metrics_df = pd.read_csv(f"Training_Sets/Training_Set_{day}/physical_level/{server.lower()}.csv", usecols=columns, sep=";")
/tmp/ipykernel_170681/2019955214.py:15: DtypeWarning: Columns (852,882,1413) have mixed types. Specify dtype option on import or set low_memory=False.
  server_metrics_df = pd.read_csv(f"Training_Sets/Training_Set_{day}/physical_level/{server.lower()}.csv", usecols=columns, sep=";")
/tmp/ipykernel_170681/2019955214.py:15: DtypeWarning: Columns (852,882,1413) have mixed types. Specify dtype option on import or set low_memory=Fals

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# Dataset Formation

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

def greedy_coreset_selection_batch(time_series_df, num_samples, batch_size=10):
    X = time_series_df[['X']].values  # valores atuais (input)
    y = time_series_df['y'].values  # valores futuros (rótulos)

    selected_indices = [np.random.randint(0, len(X))]
    remaining_indices = list(set(range(len(X))) - set(selected_indices))

    model = LinearRegression()

    model.fit(X[selected_indices], y[selected_indices])

    pbar = tqdm(total=num_samples, desc="Seleção do Coreset", unit="batch")

    while len(selected_indices) < num_samples:
        max_reduction_in_error = -np.inf
        best_candidate_indices = None

        batch_size_to_use = min(batch_size, len(remaining_indices))
        batch_candidates = np.random.choice(remaining_indices, size=batch_size_to_use, replace=False)

        temporary_coreset = X[selected_indices + list(batch_candidates)]
        temporary_labels = np.concatenate([y[selected_indices], y[batch_candidates]])

        model.fit(temporary_coreset, temporary_labels)

        error = np.mean((model.predict(X[selected_indices + list(batch_candidates)]) - y[selected_indices + list(batch_candidates)]) ** 2)

        error_reduction = error - max_reduction_in_error
        if error_reduction > max_reduction_in_error:
            max_reduction_in_error = error_reduction
            best_candidate_indices = batch_candidates

        selected_indices.extend(best_candidate_indices)
        remaining_indices = list(set(remaining_indices) - set(best_candidate_indices))

        pbar.update(batch_size_to_use)

    pbar.close()

    return time_series_df.iloc[selected_indices]

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

days = ["Day_1", "Day_2", "Day_3", "Day_4", "Day_5", "Day_6", "Day_7", "Day_8", "Day_9", "Day_10", "Day_11", "Day_12", "Day_13", "Day_14"]

global_coreset_df = pd.DataFrame()

for day in days:

    server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/{day}.xlsx")

    step = 395  # 1 minuto a frente equivale a 395 amostras a frente no timestamp

    dataset = np.array(server_metrics_df["node_load1"])

    X, y = [], []

    for i in range(len(dataset) - step):
        feature = dataset[i]
        target = dataset[i + step]
        X.append(feature)
        y.append(target)

    time_series_df = pd.DataFrame(columns=["X", "y"], data=[(X_v, y_v) for X_v, y_v in zip(X, y)])

    num_samples = 6395  # Número de amostras que você quer selecionar
    batch_size = 100

    coreset_df = greedy_coreset_selection_batch(time_series_df, num_samples, batch_size)

    global_coreset_df = pd.concat([global_coreset_df, coreset_df], ignore_index=True)

print(global_coreset_df)

Seleção do Coreset: 6400batch [00:04, 1556.87batch/s]                       
Seleção do Coreset: 6400batch [00:04, 1493.80batch/s]                       
Seleção do Coreset: 6400batch [00:04, 1385.72batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1271.15batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1072.17batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1132.66batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1113.97batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1150.22batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1223.35batch/s]                       
Seleção do Coreset: 6400batch [00:05, 1274.41batch/s]                       
Seleção do Coreset: 6400batch [00:04, 1293.54batch/s]                       
Seleção do Coreset: 6400batch [00:04, 1366.33batch/s]                       
Seleção do Coreset: 6400batch [00:04, 1343.15batch/s]                       

            X       y
0      119.00  117.89
1       97.91   99.29
2      107.83  111.56
3      115.15  114.08
4       92.39   97.70
...       ...     ...
89609   71.70   71.94
89610   71.35   72.77
89611   71.81   76.35
89612   81.03   79.27
89613   81.48   77.89

[89614 rows x 2 columns]


In [4]:
global_coreset_df.to_excel("Server_1_Training_Sets/node_load1.xlsx", index=False)